In [1]:
import pandas as pd

### 1) Attendance Data

Load the attendance.csv file

In [22]:
att = pd.read_csv('../tidy_data/untidy_data/attendance.csv', index_col=0)
att.columns = pd.to_datetime(att.columns)
att

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
Sally,P,T,T,H,P,A,T,T
Jane,A,P,T,T,T,T,A,T
Billy,A,T,A,A,H,T,P,T
John,P,T,H,P,P,T,P,P


Calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [21]:
df = att.transpose()
df.replace({'P': 1, 'T': .9, 'H': .5, 'A':0}).resample('m').mean().transpose()
df

,2018-01-31
Sally,0.7625
Jane,0.6875
Billy,0.5250
John,0.9125


### 2) Coffee Levels

- Read the coffee_levels.csv file.

In [25]:
cof = pd.read_csv('../tidy_data/untidy_data/coffee_levels.csv', index_col=0)
cof.head()

,coffee_carafe,coffee_amount
hour,,
8,x,0.816164
9,x,0.451018
10,x,0.843279
11,x,0.335533
12,x,0.898291


- Transform the data so that each carafe is in it's own column.

In [39]:
df = cof.pivot_table(values='coffee_amount', columns='coffee_carafe', index='hour')
df

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


- Is this the best shape for the data?

**No**

### 3) Cake Recipes

- Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [43]:
cake = pd.read_csv('../tidy_data/untidy_data/cake_recipes.csv')
cake

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


- Tidy the data as necessary.

In [63]:
recipe_position = cake['recipe:position'].str.extract(r'^(.?)\:(.*$)')
cake['recipe'] = recipe_position[0]
cake['position']= recipe_position[1]
df = cake.drop(columns='recipe:position').melt(id_vars=['recipe', 'position'],value_vars=['225','250','275','300'], var_name='temp',value_name='score')
df

,recipe,position,temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


- Which recipe, on average, is the best? recipe b

In [64]:
df.groupby('recipe')['score'].mean()

recipe
a    63.922201
b    76.736074
c    75.874748
d    62.864844
Name: score, dtype: float64

- Which oven temperature, on average, produces the best results? 275

In [78]:
df.groupby('temp')['score'].mean()

temp
225    71.306022
250    66.577437
275    74.886754
300    66.627655
Name: score, dtype: float64

- Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [83]:
df[df.score == df.groupby(['recipe','position','temp']).mean().max()[0]]

,recipe,position,temp,score
26,b,bottom,300,99.248541
